In [7]:
# --- IMPORTACIONES ---

# Importa librerias
import pandas as pd
import glob
import os

# Importa un diccionario con fechas de campanas de terreno
from config import field_campaigns

# Importa una funcion para identificar outliers durante campanas de terreno
from piezometer_utils import identify_campaign_outliers

In [8]:
# --- CONFIGURACIONES MODIFICABLES ---

# 1. RUTA A LOS DATOS

# Busca todos los archivos que terminen en '*_formatted.csv' en la carpeta 
# especificada y los almacena en una lista
data_path = '../../data/processed/piezometers/formatted/'
piezometer_files = glob.glob(os.path.join(data_path, '*_formatted.csv'))

# Imprime los archivos encontrados
print(f'Archivos con datos piezometricos encontrados: {len(piezometer_files)}')
print(piezometer_files)

Archivos con datos piezometricos encontrados: 6
['../../data/processed/piezometers/formatted\\SDH1PS01_formatted.csv', '../../data/processed/piezometers/formatted\\SDH1PS02_formatted.csv', '../../data/processed/piezometers/formatted\\SDH2PP01_formatted.csv', '../../data/processed/piezometers/formatted\\SDH2PS01_formatted.csv', '../../data/processed/piezometers/formatted\\SDH2PS02_formatted.csv', '../../data/processed/piezometers/formatted\\SDH2PS03_formatted.csv']


In [ ]:
# --- DEFINICION DE FUNCIONES ---

# 1. REMOCION DE DATOS ANOMALOS DURANTE CAMPANAS DE TERRENO

def remove_ouliers(df, outlier_indices):
    """Reemplaza con NaN los valores en registros identificados como outliers"""

    # Crea una copia del df
    df_cleaned = df.copy()

    # Columnas a limpiar
    columns_to_clean = ['Temperature_C', 'Depth_m', 'Static_level_masl']

    # Reemplaza los valores con NaN en los indices de outliers
    df_cleaned.loc[outlier_indices, columns_to_clean] = pd.NA

    print(f"    - Se removieron {len(outlier_indices)} registros anomalos.")

    return df_cleaned


In [10]:
# --- BUCLE DE PROCESAMIENTO ---

# Define una carpeta de salida para los datos limpios y la crea si no existe
output_cleaned_path = '../../data/processed/piezometers/cleaned/'
os.makedirs(output_cleaned_path, exist_ok=True)

# Bucle externo: itera sobre cada pozo
for file_path in piezometer_files:

    # Define el nombre de cada pozo usando el nombre de archivo
    base_name = os.path.basename(file_path)
    well_name = base_name.split('_')[0].strip()

    # Imprime un titulo que indica el nombre y archivo del pozo
    print("\n" + "="*80)
    print(f"POZO: {well_name}")
    print(f"Archivo: {base_name}")
    print("="*80)

    # Lee los datos formateados
    df_formatted = pd.read_csv(file_path, index_col='Timestamps', parse_dates=True)

    # Bucle interno: itera sobre las campanas de terreno
    for campaign_name, campaign_dates in field_campaigns.items():

        print(f"\nCampana de {campaign_name}")

        # Identifica los outliers usando la funcion de piezometer_utils.py
        outlier_indices = identify_campaign_outliers(df_formatted, campaign_dates)

        # Aplica la funcion de remocion (NaN) de outliers durante tereno
        if not outlier_indices.empty:
            df_formatted = remove_ouliers(df_formatted, outlier_indices)
        else:
            print("    - No se encontraron outliers en esta campaña.")
        
    # Genera un nombre de archivo y asigna ubicacion al dataframe limpio
    output_filename = os.path.join(output_cleaned_path, f'{well_name}_cleaned.csv')

    # Exporta el df procesado como csv
    df_formatted.to_csv(output_filename)

    # Imprime informacion sobre el nombre del output
    print(f"\nDatos limpios para {well_name} guardados en: {output_filename}")


POZO: SDH1PS01
Archivo: SDH1PS01_formatted.csv

Campana de May 2024
    - No se encontraron outliers en esta campaña.

Campana de Jul 2024
    - Se removieron 8 registros anómalos.

Campana de Sep 2024
    - Se removieron 4 registros anómalos.

Campana de Nov 2024
    - Se removieron 3 registros anómalos.

Campana de Jan 2025
    - Se removieron 4 registros anómalos.

Campana de Apr 2025
    - Se removieron 5 registros anómalos.

Campana de Jul 2025
    - No se encontraron outliers en esta campaña.

Datos limpios para SDH1PS01 guardados en: ../../data/processed/piezometers/cleaned/SDH1PS01_cleaned.csv

POZO: SDH1PS02
Archivo: SDH1PS02_formatted.csv

Campana de May 2024
    - No se encontraron outliers en esta campaña.

Campana de Jul 2024
    - No se encontraron outliers en esta campaña.

Campana de Sep 2024
    - No se encontraron outliers en esta campaña.

Campana de Nov 2024
    - Se removieron 6 registros anómalos.

Campana de Jan 2025
    - Se removieron 3 registros anómalos.

Ca